<a href="https://colab.research.google.com/github/AmritS7/Segmentation-Based-Geolocation/blob/main/ReverseGeolocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reverse Geolocation Using Segmentation-based Image Augmentation
Amritpal Singh

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd

## For Creating And Splitting Into Train/Test/Val Datasets - Don't Run

In [ ]:
#!pip install split-folders

In [ ]:
#import splitfolders
#splitfolders.ratio('Images/combined_ds', output="Images/combined_ds_split", seed=42, ratio=(.8, 0.1,0.1))

In [ ]:
#splitfolders.ratio('Images/weighted_masks', output="Images/weighted_masks_split", seed=42, ratio=(.8, 0.1,0.1))

## For Creating Augmented Datasets Based On Masks - Don't Run

In [ ]:
# import numpy as np

# def apply_to_pixels(arr, func):

#   # Use vectorized operations for efficiency
#   return np.vectorize(func, otypes=[np.float64])(arr)


# # Apply formula to each pixel and return new score to be applied to the pixel (pixel*score=newcolorintensity)
# def apply_scoring(value):
#   if value<10:
#     return ns_val
#   elif value>max_ret_threshold:
#     return max_ret_val
#   else:
#     return ((value-10)/(max_ret_threshold-10))*(1.0-min_s_val) + min_s_val

In [ ]:
# import shutil

# shutil.rmtree("Images/masked_ds_split/")

In [ ]:
# ns_val = .1 #returned value if non sig <.10
# min_s_val = .2 #minimum value if significant
# max_ret_val = 1 #maximum return value
# max_ret_threshold = 70 #threshold at which max_ret_val is returned


# directory_path = "./Images/weighted_masks_split/"

# for root, dirs, files in os.walk(directory_path):

#     dir_path = os.path.relpath(root, directory_path)
#     idx = dir_path.find("\\")
#     folder = dir_path[:idx+1]
#     country = dir_path[idx+1:]

#     for file in files: #loop through each folder in the dataset


#       if not os.path.exists(os.path.join("Images/masked_ds_split/", folder, country, file)):

#             mask = Image.open(os.path.join(directory_path, folder, country, file))
#             mask_arr = np.array(mask)
#             original = Image.open(os.path.join("Images/combined_ds_split/", folder, country, file))
#             original_arr = np.array(original)


#             score_arr = apply_to_pixels(mask_arr, apply_scoring)

#             new_arr = (score_arr[:,:,None]*original_arr).astype('uint8')
#             new_im = Image.fromarray(new_arr)

#             if not os.path.exists(os.path.join("Images/masked_ds_split/", folder, country)):
#                 os.makedirs(os.path.join("Images/masked_ds_split/", folder, country))


#             new_im.save(os.path.join(os.path.join("Images/masked_ds_split/", folder, country, file))) #save the image

## Importing Datasets From Kaggle

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d amrits7/reverse-geolocation
!unzip reverse-geolocation -d .

Streaming output truncated to the last 5000 lines.
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630005693.jpg  
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630044298.jpg  
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630080885.jpg  
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630121448.jpg  
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630138043.jpg  
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630143216.jpg  
  inflating: ./Images/weighted_masks_split/val/Hungary/canvas_1630282272.jpg  
  inflating: ./Images/weighted_masks_split/val/Iceland/2399.png  
  inflating: ./Images/weighted_masks_split/val/Iceland/canvas_1629283826.jpg  
  inflating: ./Images/weighted_masks_split/val/Iceland/canvas_1629309367.jpg  
  inflating: ./Images/weighted_masks_split/val/Iceland/canvas_1629791431.jpg  
  inflating: ./Images/weighted_masks_split/val/Iceland/canvas_1630043142.jpg  
  inflating: .

In [ ]:
os.remove("reverse-geolocation.zip")

## Applying Models

### Imports and Initial Setup

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config) #allows gpu vram growth

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv2D, MaxPool2D, BatchNormalization, Activation
from tensorflow.keras.layers import Flatten

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.random.set_seed(42)

### Universal Variable Creation

In [ ]:
batch_size = 64

steps_per_epoch = 59070//batch_size
validation_steps = 7354//batch_size

epochs = 25

In [ ]:
#create callbacks
es = EarlyStopping(monitor="val_loss", mode="min", patience=10)


reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=0.00001
)

In [ ]:
#Create ConvBlock
def ConvBlock(filters, kernel_size=(3,3), activation="relu", dropout_rate=.25):
    block = Sequential()
    block.add(Conv2D(filters=filters, kernel_size=kernel_size))
    block.add(BatchNormalization(axis=-1, trainable=True))
    block.add(Activation(activation=activation))
    block.add(MaxPool2D(pool_size=(2,2)))
    block.add(Dropout(rate=dropout_rate))
    return block

In [ ]:
#Create DenseBlock
def DenseBlock(units, activation="relu", dropout_rate=.5):
  block=Sequential()
  block.add(Dense(units=units))
  block.add(BatchNormalization(axis=-1, trainable=True))
  block.add(Activation(activation=activation))
  block.add(Dropout(rate=dropout_rate))
  return block

### Convolutional Neural Networks

#### For Base Model

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = .2,
    zoom_range = .2,
    horizontal_flip = True
)

train_generator = train_datagen.flow_from_directory(
  'Images/combined_ds_split/train',
  target_size = (224,224),
  batch_size = batch_size,
  class_mode = "categorical",
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

validation_generator = validation_datagen.flow_from_directory(
    'Images/combined_ds_split/val',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 59070 images belonging to 73 classes.
Found 7354 images belonging to 73 classes.


In [ ]:
checkpoint_filepath = "checkpoints/base"

#create checkpoint - different for each model
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only=True
)

In [ ]:
#Create model
model = Sequential()
model.add(ConvBlock(32, (3,3)))
model.add(ConvBlock(64, (3,3)))
model.add(ConvBlock(128, (3,3)))
model.add(ConvBlock(64, (3,3)))
model.add(ConvBlock(32, (3,3)))
model.add(Flatten())
model.add(DenseBlock(512))
model.add(Dense(73, activation="softmax"))

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")

In [ ]:
hist = model.fit(train_generator, steps_per_epoch=steps_per_epoch, validation_data=validation_generator, validation_steps=validation_steps, epochs=epochs, callbacks=[es, reduce_lr, model_checkpoint_callback])

Epoch 1/25
922/922 [==============================] - 584s 630ms/step - loss: 3.4180 - accuracy: 0.2042 - val_loss: 3.2678 - val_accuracy: 0.2244 - lr: 0.0010
Epoch 2/25
922/922 [==============================] - 573s 621ms/step - loss: 2.9892 - accuracy: 0.2673 - val_loss: 3.0953 - val_accuracy: 0.2466 - lr: 0.0010
Epoch 3/25
922/922 [==============================] - 521s 565ms/step - loss: 2.8149 - accuracy: 0.2961 - val_loss: 2.9667 - val_accuracy: 0.2648 - lr: 0.0010
Epoch 4/25
922/922 [==============================] - 522s 565ms/step - loss: 2.7171 - accuracy: 0.3149 - val_loss: 2.9894 - val_accuracy: 0.2499 - lr: 0.0010
Epoch 5/25
922/922 [==============================] - 519s 563ms/step - loss: 2.6338 - accuracy: 0.3323 - val_loss: 2.7605 - val_accuracy: 0.3044 - lr: 0.0010
Epoch 6/25
922/922 [==============================] - 522s 565ms/step - loss: 2.5714 - accuracy: 0.3400 - val_loss: 2.7519 - val_accuracy: 0.3148 - lr: 0.0010
Epoch 7/25
922/922 [==========================

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, None, None, 32)    1024      
                                                                 
 sequential_2 (Sequential)   (None, None, None, 64)    18752     
                                                                 
 sequential_3 (Sequential)   (None, None, None, 128)   74368     
                                                                 
 sequential_4 (Sequential)   (None, None, None, 64)    74048     
                                                                 
 sequential_5 (Sequential)   (None, None, None, 32)    18592     
                                                                 
 flatten (Flatten)           (None, None)              0         
                                                                 
 sequential_6 (Sequential)   (None, 512)               4

In [ ]:
test_datagen = ImageDataGenerator( #create generator - we create three, 1 for each test set
    rescale = 1./255
)

test_generator = test_datagen.flow_from_directory(
    'Images/combined_ds_split/test',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 7443 images belonging to 73 classes.


In [ ]:
model.load_weights("checkpoints/base") #load best model

In [ ]:
model.evaluate(test_generator, batch_size=batch_size)

117/117 [==============================] - 32s 275ms/step - loss: 2.2395 - accuracy: 0.4099


[2.239546775817871, 0.40991535782814026]

In [ ]:
test_datagen_2 = ImageDataGenerator(
    rescale = 1./255
)

test_generator_2 = test_datagen.flow_from_directory(
    'Images/masked_ds_base_split/test',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 7443 images belonging to 73 classes.


In [ ]:
model.evaluate(test_generator_2, batch_size=batch_size)

117/117 [==============================] - 28s 238ms/step - loss: 4.1011 - accuracy: 0.1198


[4.101105690002441, 0.11984414607286453]

In [ ]:
test_datagen_3 = ImageDataGenerator(
    rescale = 1./255
)

test_generator_3 = test_datagen.flow_from_directory(
    'Images/masked_ds_split/test',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 7443 images belonging to 73 classes.


In [ ]:
model.evaluate(test_generator_3, batch_size=batch_size)

117/117 [==============================] - 35s 299ms/step - loss: 3.8947 - accuracy: 0.1518


[3.89465069770813, 0.15182049572467804]

#### For Weighted Base

In [ ]:
checkpoint_filepath = "checkpoints/weighted"

#create checkpoint - different for each model
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only=True
)

In [ ]:

weighted_train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = .2,
    zoom_range = .2,
    horizontal_flip = True
)

weighted_train_generator = weighted_train_datagen.flow_from_directory(
  'Images/masked_ds_base_split/train',
  target_size = (224,224),
  batch_size = batch_size,
  class_mode = "categorical",
)

weighted_validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

weighted_validation_generator = weighted_validation_datagen.flow_from_directory(
    'Images/masked_ds_base_split/val',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 59070 images belonging to 73 classes.
Found 7354 images belonging to 73 classes.


In [ ]:
model_w = Sequential()
model_w.add(ConvBlock(32, (3,3)))
model_w.add(ConvBlock(64, (3,3)))
model_w.add(ConvBlock(128, (3,3)))
model_w.add(ConvBlock(64, (3,3)))
model_w.add(ConvBlock(32, (3,3)))
model_w.add(Flatten())
model_w.add(DenseBlock(512))
model_w.add(Dense(73, activation="softmax"))

In [ ]:
model_w.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")

In [ ]:
hist = model_w.fit(weighted_train_generator, steps_per_epoch=steps_per_epoch, validation_data=weighted_validation_generator, validation_steps=validation_steps, epochs=epochs, callbacks=[es, reduce_lr, model_checkpoint_callback])

Epoch 1/25
922/922 [==============================] - 595s 644ms/step - loss: 3.6870 - accuracy: 0.1884 - val_loss: 3.4562 - val_accuracy: 0.2049 - lr: 0.0010
Epoch 2/25
922/922 [==============================] - 574s 622ms/step - loss: 3.4713 - accuracy: 0.2031 - val_loss: 3.4131 - val_accuracy: 0.2061 - lr: 0.0010
Epoch 3/25
922/922 [==============================] - 573s 621ms/step - loss: 3.3795 - accuracy: 0.2131 - val_loss: 3.4247 - val_accuracy: 0.1953 - lr: 0.0010
Epoch 4/25
922/922 [==============================] - 571s 619ms/step - loss: 3.3349 - accuracy: 0.2173 - val_loss: 3.3083 - val_accuracy: 0.2225 - lr: 0.0010
Epoch 5/25
922/922 [==============================] - 574s 622ms/step - loss: 3.3008 - accuracy: 0.2225 - val_loss: 3.2613 - val_accuracy: 0.2226 - lr: 0.0010
Epoch 6/25
922/922 [==============================] - 570s 618ms/step - loss: 3.2732 - accuracy: 0.2227 - val_loss: 3.2834 - val_accuracy: 0.2171 - lr: 0.0010
Epoch 7/25
922/922 [==========================

In [ ]:
model_w.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_8 (Sequential)   (None, None, None, 32)    1024      
                                                                 
 sequential_9 (Sequential)   (None, None, None, 64)    18752     
                                                                 
 sequential_10 (Sequential)  (None, None, None, 128)   74368     
                                                                 
 sequential_11 (Sequential)  (None, None, None, 64)    74048     
                                                                 
 sequential_12 (Sequential)  (None, None, None, 32)    18592     
                                                                 
 flatten_1 (Flatten)         (None, None)              0         
                                                                 
 sequential_13 (Sequential)  (None, 512)              

In [ ]:
model_w.load_weights("checkpoints/weighted")

In [ ]:
model_w.evaluate(test_generator_2, batch_size=batch_size)

117/117 [==============================] - 28s 242ms/step - loss: 3.0033 - accuracy: 0.2667


[3.003345251083374, 0.2666935324668884]

In [ ]:
model_w.evaluate(test_generator, batch_size=batch_size)

117/117 [==============================] - 32s 278ms/step - loss: 3.0056 - accuracy: 0.2675


[3.0055761337280273, 0.2674996554851532]

In [ ]:
model_w.evaluate(test_generator_3, batch_size=batch_size)

117/117 [==============================] - 35s 303ms/step - loss: 2.9034 - accuracy: 0.2811


[2.9033873081207275, 0.28106945753097534]

#### Weighted Custom

In [ ]:
checkpoint_filepath = "checkpoints/weighted_custom"

#create checkpoint - different for each model
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only=True
)

In [ ]:

weighted_custom_train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = .2,
    zoom_range = .2,
    horizontal_flip = True
)

weighted_custom_train_generator = weighted_custom_train_datagen.flow_from_directory(
  'Images/masked_ds_split/train',
  target_size = (224,224),
  batch_size = batch_size,
  class_mode = "categorical",
)

weighted_custom_validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

weighted_custom_validation_generator = weighted_custom_validation_datagen.flow_from_directory(
    'Images/masked_ds_split/val',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 59070 images belonging to 73 classes.
Found 7354 images belonging to 73 classes.


In [ ]:
model_w_c = Sequential()
model_w_c.add(ConvBlock(32, (3,3)))
model_w_c.add(ConvBlock(64, (3,3)))
model_w_c.add(ConvBlock(128, (3,3)))
model_w_c.add(ConvBlock(64, (3,3)))
model_w_c.add(ConvBlock(32, (3,3)))
model_w_c.add(Flatten())
model_w_c.add(DenseBlock(512))
model_w_c.add(Dense(73, activation="softmax"))

In [ ]:
model_w_c.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")

In [ ]:
hist = model_w_c.fit(weighted_custom_train_generator, steps_per_epoch=steps_per_epoch, validation_data=weighted_custom_validation_generator, validation_steps=validation_steps, epochs=epochs, callbacks=[es, reduce_lr, model_checkpoint_callback])

Epoch 1/25
922/922 [==============================] - 660s 714ms/step - loss: 3.6162 - accuracy: 0.1879 - val_loss: 3.3107 - val_accuracy: 0.2137 - lr: 0.0010
Epoch 2/25
922/922 [==============================] - 701s 760ms/step - loss: 3.2756 - accuracy: 0.2218 - val_loss: 3.2509 - val_accuracy: 0.2245 - lr: 0.0010
Epoch 3/25
922/922 [==============================] - 689s 747ms/step - loss: 3.1252 - accuracy: 0.2422 - val_loss: 3.5564 - val_accuracy: 0.1562 - lr: 0.0010
Epoch 4/25
922/922 [==============================] - 709s 768ms/step - loss: 3.0315 - accuracy: 0.2568 - val_loss: 3.1187 - val_accuracy: 0.2421 - lr: 0.0010
Epoch 5/25
922/922 [==============================] - 707s 767ms/step - loss: 2.9450 - accuracy: 0.2730 - val_loss: 2.9400 - val_accuracy: 0.2804 - lr: 0.0010
Epoch 6/25
922/922 [==============================] - 693s 751ms/step - loss: 2.8849 - accuracy: 0.2834 - val_loss: 3.2953 - val_accuracy: 0.1798 - lr: 0.0010
Epoch 7/25
922/922 [==========================

In [ ]:
model_w_c.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_15 (Sequential)  (None, None, None, 32)    1024      
                                                                 
 sequential_16 (Sequential)  (None, None, None, 64)    18752     
                                                                 
 sequential_17 (Sequential)  (None, None, None, 128)   74368     
                                                                 
 sequential_18 (Sequential)  (None, None, None, 64)    74048     
                                                                 
 sequential_19 (Sequential)  (None, None, None, 32)    18592     
                                                                 
 flatten_2 (Flatten)         (None, None)              0         
                                                                 
 sequential_20 (Sequential)  (None, 512)             

In [ ]:
model_w_c.load_weights("checkpoints/weighted_custom")

In [ ]:
model_w_c.evaluate(test_generator_3, batch_size=batch_size)

117/117 [==============================] - 37s 315ms/step - loss: 2.5034 - accuracy: 0.3610


[2.5034079551696777, 0.36101034283638]

In [ ]:
model_w_c.evaluate(test_generator, batch_size=batch_size)

117/117 [==============================] - 36s 306ms/step - loss: 3.0419 - accuracy: 0.2487


[3.0418989658355713, 0.24869003891944885]

In [ ]:
model_w_c.evaluate(test_generator_2, batch_size=batch_size)

117/117 [==============================] - 30s 257ms/step - loss: 3.8969 - accuracy: 0.1495


[3.896921396255493, 0.14953647553920746]

### Transfer Learning With REsnet

#### Base

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
checkpoint_filepath = "checkpoints/tl_base"

#create checkpoint - different for each model
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only=True
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))

augmented_model = Sequential()

augmented_model.add(base_model)
augmented_model.add(GlobalAveragePooling2D())
augmented_model.add(DenseBlock(1024))
augmented_model.add(DenseBlock(512))
augmented_model.add(Dense(73, activation='softmax'))

In [ ]:
for i, layer in enumerate(augmented_model.layers): #freeze resnet
   if (i==0):
     layer.trainable = False
     print(f"Layer {i} (Non-trainable): {layer}")
   else:
      layer.trainable = True
      print(f"Layer {i} (Trainable): {layer}")

Layer 0 (Non-trainable): <keras.engine.functional.Functional object at 0x000001C54146F430>
Layer 1 (Trainable): <keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x000001C53C6D0DF0>
Layer 2 (Trainable): <keras.engine.sequential.Sequential object at 0x000001C53C6DF4C0>
Layer 3 (Trainable): <keras.engine.sequential.Sequential object at 0x000001C5027E2EF0>
Layer 4 (Trainable): <keras.layers.core.dense.Dense object at 0x000001C54147BF70>


In [ ]:
augmented_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")

In [ ]:
hist = augmented_model.fit(train_generator, steps_per_epoch=steps_per_epoch, validation_data=validation_generator, validation_steps=validation_steps, epochs=epochs, callbacks=[es, reduce_lr, model_checkpoint_callback])

Epoch 1/25
922/922 [==============================] - 667s 719ms/step - loss: 3.5131 - accuracy: 0.1957 - val_loss: 3.6260 - val_accuracy: 0.1790 - lr: 0.0010
Epoch 2/25
922/922 [==============================] - 577s 626ms/step - loss: 3.2791 - accuracy: 0.2234 - val_loss: 3.6618 - val_accuracy: 0.1383 - lr: 0.0010
Epoch 3/25
922/922 [==============================] - 549s 596ms/step - loss: 3.2130 - accuracy: 0.2320 - val_loss: 3.5035 - val_accuracy: 0.2113 - lr: 0.0010
Epoch 4/25
922/922 [==============================] - 537s 583ms/step - loss: 3.1711 - accuracy: 0.2369 - val_loss: 3.9055 - val_accuracy: 0.2034 - lr: 0.0010
Epoch 5/25
922/922 [==============================] - 558s 605ms/step - loss: 3.1454 - accuracy: 0.2400 - val_loss: 3.6992 - val_accuracy: 0.0744 - lr: 0.0010
Epoch 6/25
922/922 [==============================] - 559s 606ms/step - loss: 3.1202 - accuracy: 0.2432 - val_loss: 3.5147 - val_accuracy: 0.2205 - lr: 0.0010
Epoch 7/25
922/922 [==========================

In [ ]:
augmented_model.load_weights("checkpoints/tl_base")

In [ ]:
augmented_model.evaluate(test_generator, batch_size=batch_size)

117/117 [==============================] - 37s 314ms/step - loss: 2.9879 - accuracy: 0.2766


[2.987893581390381, 0.2766357660293579]

In [ ]:
augmented_model.evaluate(test_generator_2, batch_size=batch_size)

117/117 [==============================] - 31s 266ms/step - loss: 4.2903 - accuracy: 0.0364


[4.290266036987305, 0.036410048604011536]

In [ ]:
augmented_model.evaluate(test_generator_3, batch_size=batch_size)

117/117 [==============================] - 38s 327ms/step - loss: 4.1310 - accuracy: 0.0548


[4.130956172943115, 0.054816607385873795]

In [ ]:
augmented_model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 sequential_22 (Sequential)  (None, 1024)              2102272   
                                                                 
 sequential_23 (Sequential)  (None, 512)               526848    
                                                                 
 dense_8 (Dense)             (None, 73)                37449     
                                                                 
Total params: 26,254,281
Trainable params: 2,663,497
Non-trainable params: 23,590,784
_________________________________

#### Weighted Base

In [ ]:
checkpoint_filepath = "checkpoints/tl_weighted"

#create checkpoint - different for each model
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only=True
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))

augmented_model_w = Sequential()

augmented_model_w.add(base_model)  # Add the base model.
augmented_model_w.add(GlobalAveragePooling2D())
augmented_model_w.add(DenseBlock(1024))
augmented_model_w.add(DenseBlock(512))
augmented_model_w.add(Dense(73, activation='softmax'))


In [ ]:
for i, layer in enumerate(augmented_model_w.layers):
   if (i==0):
     layer.trainable = False
     print(f"Layer {i} (Non-trainable): {layer}")
   else:
      layer.trainable = True
      print(f"Layer {i} (Trainable): {layer}")

Layer 0 (Non-trainable): <keras.engine.functional.Functional object at 0x000001C5457DCCD0>
Layer 1 (Trainable): <keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x000001C5457DDC90>
Layer 2 (Trainable): <keras.engine.sequential.Sequential object at 0x000001C541455540>
Layer 3 (Trainable): <keras.engine.sequential.Sequential object at 0x000001C54185EE60>
Layer 4 (Trainable): <keras.layers.core.dense.Dense object at 0x000001C5457CCC40>


In [ ]:
augmented_model_w.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")

In [ ]:
hist = augmented_model_w.fit(weighted_train_generator, steps_per_epoch=steps_per_epoch, validation_data=weighted_validation_generator, validation_steps=validation_steps, epochs=epochs, callbacks=[es, reduce_lr, model_checkpoint_callback])

Epoch 1/25
922/922 [==============================] - 656s 708ms/step - loss: 3.6611 - accuracy: 0.1874 - val_loss: 3.9290 - val_accuracy: 0.0448 - lr: 0.0010
Epoch 2/25
922/922 [==============================] - 648s 703ms/step - loss: 3.5100 - accuracy: 0.2002 - val_loss: 3.8685 - val_accuracy: 0.0362 - lr: 0.0010
Epoch 3/25
922/922 [==============================] - 639s 693ms/step - loss: 3.4665 - accuracy: 0.2028 - val_loss: 3.7421 - val_accuracy: 0.0796 - lr: 0.0010
Epoch 4/25
922/922 [==============================] - 630s 683ms/step - loss: 3.4483 - accuracy: 0.2038 - val_loss: 3.8020 - val_accuracy: 0.0561 - lr: 0.0010
Epoch 5/25
922/922 [==============================] - 638s 692ms/step - loss: 3.4341 - accuracy: 0.2042 - val_loss: 3.8467 - val_accuracy: 0.2042 - lr: 0.0010
Epoch 6/25
922/922 [==============================] - 642s 697ms/step - loss: 3.4243 - accuracy: 0.2055 - val_loss: 3.7419 - val_accuracy: 0.2053 - lr: 0.0010
Epoch 7/25
922/922 [==========================

In [ ]:
augmented_model_w.load_weights("checkpoints/tl_weighted")

In [ ]:
augmented_model_w.evaluate(test_generator_2, batch_size=batch_size)

117/117 [==============================] - 30s 258ms/step - loss: 3.4459 - accuracy: 0.2099


[3.44594407081604, 0.209861621260643]

In [ ]:
augmented_model_w.evaluate(test_generator, batch_size=batch_size)

117/117 [==============================] - 36s 304ms/step - loss: 3.5564 - accuracy: 0.1786


[3.5564236640930176, 0.17855703830718994]

In [ ]:
augmented_model_w.evaluate(test_generator_3, batch_size=batch_size)

117/117 [==============================] - 38s 321ms/step - loss: 3.4127 - accuracy: 0.2128


[3.41271710395813, 0.21281741559505463]

#### Weighted Custom

In [ ]:
checkpoint_filepath = "checkpoints/tl_weighted_custom"

#create checkpoint - different for each model
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor = "val_accuracy",
    mode = "max",
    save_best_only=True
)

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))

augmented_model_w_c = Sequential()

augmented_model_w_c.add(base_model)  # Add the base model.
augmented_model_w_c.add(GlobalAveragePooling2D())
augmented_model_w_c.add(DenseBlock(1024))
augmented_model_w_c.add(DenseBlock(512))
augmented_model_w_c.add(Dense(73, activation='softmax'))

In [ ]:
for i, layer in enumerate(augmented_model_w_c.layers):
   if (i==0):
     layer.trainable = False
     print(f"Layer {i} (Non-trainable): {layer}")
   else:
      layer.trainable = True
      print(f"Layer {i} (Trainable): {layer}")

Layer 0 (Non-trainable): <keras.engine.functional.Functional object at 0x000001C5462130A0>
Layer 1 (Trainable): <keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x000001C546213CD0>
Layer 2 (Trainable): <keras.engine.sequential.Sequential object at 0x000001C5418B1630>
Layer 3 (Trainable): <keras.engine.sequential.Sequential object at 0x000001C5457DE170>
Layer 4 (Trainable): <keras.layers.core.dense.Dense object at 0x000001C546211930>


In [ ]:
augmented_model_w_c.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer="adam")

In [ ]:
hist = augmented_model_w_c.fit(weighted_custom_train_generator, steps_per_epoch=steps_per_epoch, validation_data=weighted_custom_validation_generator, validation_steps=validation_steps, epochs=epochs, callbacks=[es, reduce_lr, model_checkpoint_callback])

Epoch 1/25
922/922 [==============================] - 734s 792ms/step - loss: 3.6660 - accuracy: 0.1832 - val_loss: 3.9929 - val_accuracy: 0.0408 - lr: 0.0010
Epoch 2/25
922/922 [==============================] - 711s 771ms/step - loss: 3.4944 - accuracy: 0.2010 - val_loss: 3.8257 - val_accuracy: 0.0942 - lr: 0.0010
Epoch 3/25
922/922 [==============================] - 716s 777ms/step - loss: 3.4470 - accuracy: 0.2033 - val_loss: 3.6386 - val_accuracy: 0.2041 - lr: 0.0010
Epoch 4/25
922/922 [==============================] - 721s 781ms/step - loss: 3.4246 - accuracy: 0.2052 - val_loss: 3.6360 - val_accuracy: 0.2005 - lr: 0.0010
Epoch 5/25
922/922 [==============================] - 704s 763ms/step - loss: 3.4096 - accuracy: 0.2052 - val_loss: 3.7505 - val_accuracy: 0.2023 - lr: 0.0010
Epoch 6/25
922/922 [==============================] - ETA: 0s - loss: 3.3989 - accuracy: 0.2060
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
922/922 [========================

In [ ]:
augmented_model_w_c.load_weights("checkpoints/tl_weighted_custom")

In [ ]:
augmented_model_w_c.evaluate(test_generator_3, batch_size=batch_size)

117/117 [==============================] - 37s 311ms/step - loss: 3.3521 - accuracy: 0.2173


[3.3520596027374268, 0.21725110709667206]

In [ ]:
augmented_model_w_c.evaluate(test_generator, batch_size=batch_size)

117/117 [==============================] - 33s 281ms/step - loss: 3.4502 - accuracy: 0.1976


[3.450197219848633, 0.19763536751270294]

In [ ]:
augmented_model_w_c.evaluate(test_generator_2, batch_size=batch_size)

117/117 [==============================] - 29s 245ms/step - loss: 3.4329 - accuracy: 0.2057


[3.432913064956665, 0.20569662749767303]

## Testing On IMGPS3k

In [ ]:
countries = ['United States', 'Japan', 'France', 'Brazil', 'Australia',
       'United Kingdom', 'Russia', 'Argentina', 'Poland', 'Canada', 'Mexico',
       'South Africa', 'Finland', 'Spain', 'Thailand', 'Italy', 'Sweden',
       'Turkey', 'Nigeria', 'Germany', 'Portugal', 'Malaysia', 'Netherlands',
       'Singapore', 'Norway', 'South Korea', 'Austria', 'India', 'Indonesia',
       'Colombia', 'Ireland', 'Romania', 'New Zealand', 'Taiwan', 'Czechia',
       'Israel', 'Chile', 'Peru', 'Belgium', 'Switzerland', 'Bulgaria',
       'Kenya', 'Philippines', 'Sri Lanka', 'Botswana', 'Denmark', 'Mongolia',
       'Lithuania', 'Greece', 'Ukraine', 'Slovakia', 'Bolivia', 'Hungary',
       'Bangladesh', 'Croatia', 'Senegal', 'Ghana', 'Cambodia', 'Uruguay',
       'Latvia', 'Ecuador', 'Palestine', 'Estonia', 'Serbia', 'Tunisia',
       'Guatemala', 'Hong Kong', 'Kyrgyzstan', 'Jordan', 'Slovenia', 'Iceland',
       'Lesotho', 'Eswatini']

In [ ]:
for i in countries: #create empty folders that dont exist to match our 73 classes
    if not os.path.exists(os.path.dirname(os.path.join("Images/im2gps3k", i, "1"))):
         os.makedirs(os.path.dirname(os.path.join("Images/im2gps3k", i, "1")))

In [ ]:
im2gps3k_datagen = ImageDataGenerator(
    rescale = 1./255
)

im2gps3k_generator = im2gps3k_datagen.flow_from_directory(
    'Images/im2gps3k',
    target_size = (224,224),
    batch_size = batch_size,
    class_mode="categorical"
)

Found 2504 images belonging to 73 classes.


In [ ]:
model.evaluate(im2gps3k_generator, batch_size=batch_size)

40/40 [==============================] - 21s 532ms/step - loss: 3.8621 - accuracy: 0.1282


[3.8620855808258057, 0.1281948834657669]

In [ ]:
model_w.evaluate(im2gps3k_generator, batch_size=batch_size)

40/40 [==============================] - 11s 264ms/step - loss: 3.7619 - accuracy: 0.1613


[3.7618825435638428, 0.1613418459892273]

In [ ]:
model_w_c.evaluate(im2gps3k_generator, batch_size=batch_size)

40/40 [==============================] - 11s 257ms/step - loss: 3.9259 - accuracy: 0.1090


[3.9259469509124756, 0.10902556031942368]

In [ ]:
augmented_model.evaluate(im2gps3k_generator, batch_size=batch_size)

40/40 [==============================] - 13s 273ms/step - loss: 3.7413 - accuracy: 0.1430


[3.7413060665130615, 0.1429712474346161]

In [ ]:
augmented_model_w.evaluate(im2gps3k_generator, batch_size=batch_size)

40/40 [==============================] - 12s 263ms/step - loss: 3.7214 - accuracy: 0.1286


[3.7214202880859375, 0.12859424948692322]

In [ ]:
augmented_model_w_c.evaluate(im2gps3k_generator, batch_size=batch_size)

40/40 [==============================] - 12s 260ms/step - loss: 3.6601 - accuracy: 0.1458


[3.66011643409729, 0.14576677978038788]

## Comparing Performance to StreetCLIP

In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch

c:\Users\Amritpal\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{str(device).upper()} is available.")

CUDA is available.


In [ ]:
model = CLIPModel.from_pretrained("geolocal/StreetCLIP")
processor = CLIPProcessor.from_pretrained("geolocal/StreetCLIP")

model.to(device)

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05,

In [ ]:
def find_locations(images, locations, labels): #Manually loop through all images and calculate accuracy
    counter = 0
    for i, image_path in enumerate(images): #goes through all images in array
        image = Image.open(image_path)
        inputs = processor(
            text=labels, images=image, return_tensors="pt", padding=True
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        logits_per_image = outputs.logits_per_image
        prediction = logits_per_image.softmax(dim=1)

        scores = {labels[i]: float(prediction[0][i].item()) for i in range(len(labels))}
        sorted_countries = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        sorted_scores = dict(sorted_countries)
        pred = next(iter(sorted_scores.items()))[0] #get country with highest accuracy
        if pred == locations[i]: #if the prediction matches the actual country, increase counter by 1
            counter+=1

    return counter/len(images)

In [ ]:
#manually do for each different test set - for original
image_paths = []
image_countries = []

import os
for folder, _, files in os.walk("Images/combined_ds_split/test"):
    country_path = folder.split("/")[-1]
    country_name = country_path.split("\\")[-1]

    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):
            file_path = os.path.join(folder, file)
            image_paths.append(file_path)
            image_countries.append(country_name)

In [ ]:
acc = find_locations(image_paths, image_countries, countries)
print(acc)

0.714496842670966


In [ ]:
image_paths_2 = []

#for weighted base
for folder, _, files in os.walk("Images/masked_ds_base_split/test"):
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):  # Add more extensions if needed
            file_path = os.path.join(folder, file)
            image_paths_2.append(file_path)

In [ ]:
acc2 = find_locations(image_paths_2, image_countries, countries)
print(acc2)

0.21631062743517399


In [ ]:
image_paths_3 = []

#for weighted custom
for folder, _, files in os.walk("Images/masked_ds_split/test"):
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):  # Add more extensions if needed
            file_path = os.path.join(folder, file)
            image_paths_3.append(file_path)

In [ ]:
acc3 = find_locations(image_paths_3, image_countries, countries)
print(acc3)

0.47440548166062074


In [ ]:
image_paths_im2gps3k = []
image_countries_im2gps3k = []

#for im2gps3k

for folder, _, files in os.walk("Images/im2gps3k"):
    country_path = folder.split("/")[-1]
    country_name = country_path.split("\\")[-1]
    for file in files:
        if file.endswith((".jpg", ".jpeg", ".png")):  # Add more extensions if needed
            file_path = os.path.join(folder, file)
            image_paths_im2gps3k.append(file_path)
            image_countries_im2gps3k.append(country_name)

In [ ]:
accim2gps3k = find_locations(image_paths_im2gps3k, image_countries_im2gps3k, countries)
print(accim2gps3k)

0.5630990415335463


## Basic Visualization

In [ ]:
original = Image.open("./Images/combined_ds_split/train/United Kingdom/22100.png")
mask = Image.open("./Images/weighted_masks_split/train/United Kingdom/22100.png")
weighted_base = Image.open("./Images/masked_ds_base_split/train/United Kingdom/22100.png")
weighted_custom = Image.open("./Images/masked_ds_split/train/United Kingdom/22100.png")

# Convert images to NumPy arrays
original = np.array(original)
mask = np.array(mask)
weighted_base = np.array(weighted_base)
weighted_custom = np.array(weighted_custom)

fig, axis = plt.subplots(2, 2, figsize=(100, 100))
fig.subplots_adjust(wspace=0.1, hspace=0.05)

plt.subplot(2, 2, 1)
plt.imshow(original)
plt.axis('off')
plt.title("Original", fontsize=120)

plt.subplot(2, 2, 2)
plt.imshow(mask, cmap="gray", vmin=0, vmax=255)
plt.axis('off')
plt.title("Mask", fontsize=120)

plt.subplot(2, 2, 3)
plt.imshow(weighted_base)
plt.axis('off')
plt.title("Weighted Base", fontsize=120)

plt.subplot(2, 2, 4)
plt.imshow(weighted_custom)
plt.axis('off')
plt.title("Weighted Custom", fontsize=120)

plt.show()